## Smock_Data

In [1]:
import os
import json
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize

In [62]:
train_data_path = '../GoogleAnalyticsCustomerRevenuePrediction/data/train.csv'

In [65]:
def load_df(csv_path='../GoogleAnalyticsCustomerRevenuePrediction/data/train.csv', nrows=None):
    JSON_COLUMNS = ['device', 'geoNetwork', 'totals', 'trafficSource']
    
    df = pd.read_csv(csv_path, 
                     converters={column: json.loads for column in JSON_COLUMNS}, 
                     dtype={'fullVisitorId': 'str'}, # Important!!
                     nrows=nrows)
    
    for column in JSON_COLUMNS:
        column_as_df = json_normalize(df[column])
        column_as_df.columns = [f"{column}.{subcolumn}" for subcolumn in column_as_df.columns]
        df = df.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True)
    print(f"Loaded {os.path.basename(csv_path)}. Shape: {df.shape}")
    return df

print(os.listdir("../GoogleAnalyticsCustomerRevenuePrediction/data"))

['sample_submission.csv', 'test.csv', 'train.csv', 'zip']


In [66]:
%%time
df_small = load_df(nrows=20000)

Loaded train.csv. Shape: (20000, 54)
Wall time: 2.86 s


In [67]:
%%time
df_train = load_df()
df_test = load_df("../GoogleAnalyticsCustomerRevenuePrediction/data/test.csv")

Loaded train.csv. Shape: (903653, 55)
Loaded test.csv. Shape: (804684, 53)
Wall time: 5min 11s


In [68]:
%%time
df_train.to_csv("../GoogleAnalyticsCustomerRevenuePrediction/data/train-flattened.csv", index=False)
df_test.to_csv("../GoogleAnalyticsCustomerRevenuePrediction/data/test-flattened.csv", index=False)

Wall time: 2min 45s
